In [2]:
import os, sys
import json
import pandas as pd

In [3]:
srcFiles = os.listdir('./lodash/')

In [4]:
for item in srcFiles:
    inputPath = os.path.join('./lodash/', item)
    outputPath = os.path.join('./lodash_ob/', item)
    os.system(f'node pjs-obfuscator.js {inputPath} > {outputPath}')

Deobfus Jsjiami

In [9]:
import simulation
import importlib
import re

In [52]:
importlib.reload(simulation)

<module 'simulation' from '/Users/chentianyu/Documents/temp/major/simulation.py'>

In [25]:
temp_folder = './temp_json/'
input_folder = './lodash_jsjiami/'
output_folder = './lodash_deobs/'

srcFiles =[item for item in os.listdir(input_folder) if not item.startswith('.') and item.endswith('.js')]

In [33]:
importReg = 'import[^\n]*'
exportReg = 'export default[^\n]*;'

In [ ]:
fails = ['each.js', 'debounce.js', 'first.js', 'eachRight.js']

counter = 0
for file in srcFiles:
    counter = counter + 1
    if counter < 150:
        continue
    srcPath = os.path.join(input_folder, file)
    with open(srcPath, 'r') as f:
        src = f.read()
    imp = '\n'.join(re.findall(importReg, src))
    exp = '\n'.join(re.findall(exportReg, src))
#     print(imp, exp)
    
    cur = re.sub(importReg, '', src)
    cur = re.sub(exportReg, '', cur)

    curPath = os.path.join(temp_folder, file)
    print(curPath)
    with open(curPath, 'w') as f:
        f.write(cur)
    targetPath = os.path.join(output_folder, file)

#     print('cur', cur)

    if len(cur) < 10 or file in fails:
        os.system(f'cp {srcPath} {targetPath}')
        continue

    simulation.simulate(file, 'temp_json/', output_folder)
    with open(targetPath, 'r') as f:
        src = f.read()
    with open(targetPath, 'w') as f:
        f.write(f'{imp}\n{src}\n{exp}')
#     print(imp, exp, cur)
#     break

J-ob deobfus

In [185]:
import sim_ob

In [201]:
importlib.reload(sim_ob)

<module 'sim_ob' from '/Users/chentianyu/Documents/temp/major/sim_ob.py'>

In [183]:
temp_folder = './temp_job/'
input_folder = './lodash_ob/'
output_folder = './lodash_rec_ob_raw/'

srcFiles =[item for item in os.listdir(input_folder) if not item.startswith('.') and item.endswith('.js')]

In [171]:
importReg = 'import[^\n]*'
exportReg = 'export default[^\n]*;'

In [ ]:
counter = 0
for file in srcFiles:
    counter = counter + 1
#     if counter > 10:
#         break
#     if file != 'upperFirst.js':
#         continue
    if counter < 60:
        continue
    srcPath = os.path.join(input_folder, file)
    with open(srcPath, 'r') as f:
        src = f.read()
    imp = '\n'.join(re.findall(importReg, src))
    exp = '\n'.join(re.findall(exportReg, src))
    print(imp, exp)
    
    cur = re.sub(importReg, '', src)
    cur = re.sub(exportReg, '', cur)

    curPath = os.path.join(temp_folder, file)
    print(curPath)
    with open(curPath, 'w') as f:
        f.write(cur)
    targetPath = os.path.join(output_folder, file)

#     print('cur', cur)

    if len(cur) < 10 or file in fails:
        os.system(f'cp {srcPath} {targetPath}')
        continue

    sim_ob.simulate(file, temp_folder, output_folder)
    with open(targetPath, 'r') as f:
        src = f.read()
    with open(targetPath, 'w') as f:
        f.write(f'{imp}\n{src}\n{exp}')
#     print(f'{imp}\n{src}\n{exp}')
#     print(imp, exp, cur)
#     break

In [64]:
inputFileName = 'floor.js'
inputPath = '.'
outputPath = 'temp_job'

In [127]:
inputFile = os.path.join(inputPath, inputFileName)
#     print('inputFile', inputFile)
#     inputFile = './jsdata/rand_jsjiami/' + inputFileName
# file = open(inputFile)
# if not os.path.exists(inputFile + '-fun'):
#     os.mkdir(inputFile + '-fun')
srcFile = os.popen('node pparse.js ' + inputFile, 'r')# Not sure ...'r', 1)
syntax = json.loads(srcFile.read())
srcFile.close()

In [111]:
with open('temp_job/t1.json', 'w') as f:
    json.dump(syntax, f)

In [167]:
def DFS_alias(node, illgalFuns, decs):
    if type(node) == dict:
        for key in node:
            DFS_alias(node[key], illgalFuns, decs)
    elif type(node) == list:
        to_remove = []
        for item in node:
            if item['type'] == 'VariableDeclarator' and item['id']['type'] == 'Identifier'\
                and item['id']['name'].startswith('_0x') and item['init']['type'] == 'Identifier'\
                and item['init']['name'] in illgalFuns:
                illgalFuns.add(item['id']['name'])
                decs.append(item)
                to_remove.append(item)
            else:
                DFS_alias(item, illgalFuns, decs)
        for item in to_remove:
            node.remove(item)

def DFS_clear(node):
    if type(node) == dict:
        for key in node:
            DFS_clear(node[key])
    elif type(node) == list:
        to_remove = []
        for item in node:
            if item['type'] == 'VariableDeclaration' and len(item['declarations']) == 0:
                to_remove.append(item)
            else:
                DFS_clear(item)
        for item in to_remove:
            node.remove(item)

In [104]:
decs = []
DFS_alias(syntax['body'], illgalFuns, decs)

In [105]:
decs

[]

In [128]:
newProgram = {'type': 'Program', 'body': [], 'sourceType': 'script'}
illgalFuns = set()
for item in syntax['body']:
    if item['type'] == 'ExpressionStatement':
        newProgram['body'].append(item)
    
#     if item['type'] == 'VariableDeclaration':
#         target_dec = {"type": "VariableDeclaration", "declarations": [], 'kind': "const"}
#         for dec in item['declarations']:
#             if dec['type'] == 'VariableDeclarator' and dec['id']['type'] == 'Identifier'\
#                 and dec['id']['name'].startswith('_0x'): 
#                 continue
#                 target_dec['declarations'].append(dec)
#                 illgalFuns.add(dec['id']['name'])
#         if len(target_dec['declarations']) > 0:
#             newProgram['body'].append(target_dec)
    if item['type'] == 'FunctionDeclaration' and item['id']['type'] == 'Identifier'\
        and item['id']['name'].startswith('_0x'): 
        illgalFuns.add(item['id']['name'])
        newProgram['body'].append(item)

decs = []
for item in syntax['body']:
    if item not in newProgram['body']:
        DFS_alias(item, illgalFuns, decs)
        
target_dec = {"type": "VariableDeclaration", "declarations": decs, 'kind': "const"}
if len(target_dec['declarations']) > 0:
    newProgram['body'].append(target_dec)



In [129]:
decs, illgalFuns

([{'type': 'VariableDeclarator',
   'id': {'type': 'Identifier', 'name': '_0x2e3efa'},
   'init': {'type': 'Identifier', 'name': '_0x1048'}}],
 {'_0x1048', '_0x1598', '_0x2e3efa'})

In [130]:
illegalName = os.path.join('./temp_job', inputFileName + '.json')
with open(illegalName, 'w') as f:
    json.dump(newProgram, f)
dstFile = os.popen('node ./pregenerate.js ' + illegalName, 'r')
dstRes = dstFile.read()

In [139]:
for item in newProgram['body']:
    if item in syntax['body']:
        syntax['body'].remove(item)

In [161]:
def DFS_replace(node, nodeName, jsFun):
    if type(node) == dict:
        for key in node.keys():
            next = node.get(key)
            if type(next) == dict:
                token = next.get('type')
                if token == 'CallExpression' and next.get('callee').get('type') == 'Identifier'\
                and next.get('callee').get('name') in nodeName:
                    funName = next.get('callee').get('name')
                    parameters = []
                    if 'arguments' in next.keys():
                        for argument in next.get('arguments'):
                            parameters.append(argument.get('value'))
                    # print("parameters", parameters, jsFun)
#                     res = jsFun(parameters[0], parameters[1])
                    curFun = f'{jsFun}\n{funName}{tuple(parameters)};'
                    curFun = curFun.replace(',)', ')')
#                     print("curFun", curFun)
                    res = js2py.eval_js(curFun)
                    print('res', res)
                    newNode = {'type' : 'Literal', 'value' : res, 'raw' : '\'' + res + '\''}
                    node[key] = newNode
                    continue
            DFS_replace(next, nodeName, jsFun)
    elif type(node) == list:
        for i in range(len(node)):
            next = node[i]
            if type(next) == dict:
                token = next.get('type')
                if token == 'CallExpression' and next.get('callee').get('type') == 'Identifier'\
                and next.get('callee').get('name') in nodeName:
                    funName = next.get('callee').get('name')
                    parameters = []
                    if 'arguments' in next.keys():
                        for argument in next.get('arguments'):
                            parameters.append(argument.get('value'))
#                     print("parameters", parameters)
                    curFun = f'{jsFun}\n{funName}{tuple(parameters)};'
                    curFun = curFun.replace(',)', ')')
#                     print("curFun", curFun)
                    res = js2py.eval_js(curFun)
                    print('eval res', res)
#                     res = jsFun(parameters[0], parameters[1])
                    # print("res", res)
                    newNode = {'type' : 'Literal', 'value' : res, 'raw' : '\'' + res + '\''}
                    node[i] = newNode
                    continue
            DFS_replace(node[i], nodeName, jsFun)

In [141]:
print(syntax)

{'type': 'Program', 'body': [{'type': 'VariableDeclaration', 'declarations': [], 'kind': 'const'}, {'type': 'VariableDeclaration', 'declarations': [{'type': 'VariableDeclarator', 'id': {'type': 'Identifier', 'name': 'floor'}, 'init': {'type': 'CallExpression', 'callee': {'type': 'Identifier', 'name': '_0x487f26'}, 'arguments': [{'type': 'CallExpression', 'callee': {'type': 'Identifier', 'name': '_0x2e3efa'}, 'arguments': [{'type': 'Literal', 'value': 338, 'raw': '0x152'}]}]}}], 'kind': 'const'}], 'sourceType': 'script'}


In [156]:
tuple([1])

(1,)

In [162]:
DFS_replace(syntax, illgalFuns, dstRes)

In [168]:
DFS_clear(syntax)

In [169]:
    outputFile = os.path.join(outputPath, inputFileName)
#     outputFile = './jsdata/rand_jsjiami_recover/' + inputFileName

    newFile = open(outputFile + '-fixed.json', 'w')
    newFile.write(json.dumps(syntax, indent = 4))
    newFile.close()
    # print(syntax)
    os.system('node ./regenerate.js ' + outputFile + '-fixed.json ' + outputFile)



0

In [166]:
outputPath, outputFile

('temp_job', 'temp_job/floor.js')

In [122]:
import js2py

In [126]:
js2py.eval_js('function f(){return 1;}; const g = f; g();')

1

In [132]:
js2py.eval_js(f'{dstRes}\n _0x2e3efa(0x152);')

'floor'

In [89]:
illgalFuns

{'_0x1048', '_0x1598'}